In [ ]:
from utils.model import (
    HParams,
    Path,
    prepare_data_for_training,
    torch,
    NRMSModel,
    train_and_evaluate,
    evaluate_model,
)
import utils.model
import sys
import os
from importlib import reload

# Get the current directory
current_dir = os.getcwd()

# Append the relative path to the utils folder
sys.path.append(os.path.join(current_dir, "utils"))


reload(utils.model)

In [ ]:
# Setting hyperparameters
hparams = HParams()
hparams.data_fraction = 0.01
hparams.batch_size = 32
print("Hyperparameters:", hparams)

# Preprocessing and Loading Data
# Data loading
PATH = Path("~/Git Repositories/ebnerd-benchmark/data").expanduser()
DATASPLIT = "ebnerd_small"

word_embeddings, train_loader, val_loader = prepare_data_for_training(
    hparams, PATH, DATASPLIT
)

# Initialize and train model
print("Training model with ", hparams)
device = torch.device("cunda" if torch.cuda.is_available() else "cpu")
model = NRMSModel(hparams, word_embeddings)

model = train_and_evaluate(
    device,
    model,
    train_loader,
    val_loader,
    num_epochs=hparams.epochs,
    learning_rate=1e-3,
    patience=3,
)

# Evaluate model
metrics = evaluate_model(model, val_loader, device)
print("\nValidation Metrics:")
for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")

Hyperparameters: 
 title_size: 30
 head_num: 16
 head_dim: 16
 attention_hidden_dim: 200
 dropout: 0.2
 batch_size: 32
 verbose: False
 data_fraction: 0.01
 sampling_nratio: 4
 history_size: 20
 epochs: 1
 transformer_model_name: facebookai/xlm-roberta-base
Train samples: 2040
Validation samples: 302
Training model with  
 title_size: 30
 head_num: 16
 head_dim: 16
 attention_hidden_dim: 200
 dropout: 0.2
 batch_size: 32
 verbose: False
 data_fraction: 0.01
 sampling_nratio: 4
 history_size: 20
 epochs: 1
 transformer_model_name: facebookai/xlm-roberta-base


Validation: 100%|██████████| 10/10 [00:01<00:00,  6.74batch/s]


Epoch 1/1, Train Loss: 1.6020, Val Loss: 1.5976, Val AUC: 0.5860, Improvement from Previous Epoch: 0.5860
Checkpoint saved to: checkpoints/nrms_checkpoint_1.pth

Validation Metrics:
auc: 0.5860
